In [ ]:
import numpy as np
import pandas as pd

def radar(sheet_name, index):
    # データ読み込み
    zemidf = pd.read_excel('./data.xlsx', sheet_name='data', index_col=0)
    studentdf = pd.read_excel('./data.xlsx', sheet_name='data')
    ratiodf = pd.read_excel('./data.xlsx', sheet_name=sheet_name)

    student_name = studentdf.loc[index, 'student']
    student_preference = studentdf.loc[index, 'preference']
    answer = studentdf.iloc[index]

    total_scores = []
    zemi_values =[]

    # スコア計算
    for i in zemidf.index:

        total_score = 0
        answer_count = 0
        zemi_value_sum = 0

        for _, row in ratiodf.iterrows():
            question = row["question"]
            if answer[question] >= 3:
                answer_count += 1

        #ゼロ除算の防止
        if answer_count == 0:
            answer_count = 1

        answer_count = np.sqrt(answer_count)

        for _, row in ratiodf.iterrows():
            question = row["question"]
            ratio = row["ratio"]

            student_answer = answer[question]
            zemi_value = zemidf.loc[i, question]
            zemi_value_sum += zemi_value

            if zemi_value < 2:
                continue

            # 一致した場合のボーナスポイント
            if student_answer == zemi_value:
                total_score += 5

            difference = ((student_answer - zemi_value) ** 2) / answer_count
            if difference <= 1:
                difference *= 0.9

            total_score += difference * ratio

        zemidf.loc[i, "score"] = total_score
        total_scores.append(total_score)

        
    # スコア正規化
    scores = np.array(total_scores)
    zemis = np.array(zemi_values)

    if len(scores) > 1:
        mean_score = np.mean(scores)
        std_dev = np.std(scores)

        if std_dev != 0:
            z_scores = (scores - mean_score) / std_dev
            normalized_scores = 50 + (z_scores * 10)
            normalized_scores = np.clip(normalized_scores, 0, 100)

            if all(normalized_scores) < 70:
                normalized_scores = normalized_scores + 20
        else:
            normalized_scores = np.full_like(scores, 50)
    else:
        normalized_scores = scores


    # 完全一致なら100点
    for idx, (i, row) in enumerate(zemidf.iterrows()):
        all_match = all(
            question in answer and str(answer[question]) != "" and str(answer[question]) == str(row[question])
            for question in ratiodf["question"]
            if question in row
        )
        
        if all_match:
            normalized_scores[idx] = 100
    
    # スコアを保存
    zemidf["normalized_score"] = normalized_scores

    # 上位10件を表示
    top10 = zemidf.sort_values('normalized_score', ascending=False)
    columns_to_display = ["score", "normalized_score"]
    display(top10.loc[top10.index[:10], columns_to_display])
    print(student_name, student_preference, answer_count)